In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
def paginas(link):
    parser_pag = parser(link)

    paginas = parser_pag.xpath('//a[@class="page-numbers"]//@href')
    paginas.append(link)

    return paginas

In [4]:
pag = 'https://www.senacyt.gob.pa/category/convocatorias/'

def links_proyectos(link):

    links_proyectos_final = []
    paginacion = paginas(link)

    for pag in paginacion:
        links = parser(pag)
        links_proyectos = links.xpath('//div[@class="post_content"]//a/@href')
        links_proyectos2 = links.xpath('//div[@class="post_no_thumb_content"]//a/@href')

        links_proyectos += links_proyectos2


        for url in links_proyectos:
            if 'author' not in url:
                links_proyectos_final.append(url)

    return links_proyectos_final


def titulo_proyecto(link_proyecto):
    link_proyecto = link_proyecto
    parser_proyecto = parser(link_proyecto)

    titulo = parser_proyecto.xpath('//h1/text()')
    
    return titulo[0]


def descripcion_proyecto(link_proyecto):

    link_proyecto = link_proyecto

    parser_proyecto = parser(link_proyecto)
    descripcion = parser_proyecto.xpath('//div[@class="post_content"]//p//text()')
    descripcion_completa = ''

    for i in descripcion:
        descripcion_completa = descripcion_completa + i.strip() + ' '

    return descripcion_completa.strip()

def pdfs_proyecto(link_proyecto):

    link_proyecto = link_proyecto

    parser_proyecto = parser(link_proyecto)
    pdfs_pag = parser_proyecto.xpath('//div[@class="su-table su-table-responsive su-table-alternate"]//a/@href')

    pdfs = ''
    for i in pdfs_pag:
        if i.endswith('.pdf'):
            pdfs = pdfs + i.strip('"') + ', '

    return pdfs.strip(', ')

In [5]:
def panama():
    titulos = []
    descripciones = []
    titulos = []
    pdfs = []


    pag = 'https://www.senacyt.gob.pa/category/convocatorias/'
    links_proyectos_final = links_proyectos(pag)

    for link_proyecto in links_proyectos_final:
        
        try:
            titulo = titulo_proyecto(link_proyecto)
        except:
            totulo = ''
        
        try:
            descripcion = descripcion_proyecto(link_proyecto)
        except:
            descripcion = ''
            
        try:
            pdf = pdfs_proyecto(link_proyecto)
        except:
            pdf = ''

        titulos.append(titulo)
        descripciones.append(descripcion)
        pdfs.append(pdf)

        sleep(random.randrange(1, 3))
        
    panama = pd.DataFrame()
    panama['Titulos'] = titulos
    panama['Descripcion'] = descripciones
    panama['PDFs'] = pdfs
    panama['Link'] = links_proyectos_final

    return panama


In [6]:
panama = panama()
panama.to_excel('Panama.xlsx')

/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.senacyt.gob.pa'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,Titulos,Descripcion,PDFs,Link
0,CONVOCATORIA PÚBLICA DE DIPLOMADO EN MATEMÁTICA,"DIRIGIDA A: Docentes de educación primaria, co...",https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/convocatoria-public...
1,CONVOCATORIA PÚBLICA DE ESPECIALIZACIÓN EN DID...,"DIRIGIDA A: profesores de educación media, con...",https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/convocatoria-public...
2,MAESTRÍA EN ÁREAS ESPECÍFICAS DEL CONOCIMIENTO...,DIRIGIDA A: Personas de nacionalidad panameña ...,https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/maestria-en-areas-e...
3,CONVOCATORIA DE SEGUIMIENTO DE VACUNAS PARA CO...,DIRIGIDO A: Esta convocatoria hace un llamado ...,https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/convocatoria-de-seg...
4,CONVOCATORIA PÚBLICA PARA EL FOMENTO A LA INVE...,OBJETIVO: Fortalecer la capacidad nacional de ...,https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/convocatoria-public...
...,...,...,...,...
243,CONVOCATORIA PÚBLICA PARA FORTALECER CONOCIMIE...,"DIRIGIDA A: Sociedades anónimas, entidades Aca...",https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/convocatoria-public...
244,DOCTORADO DE INVESTIGACIÓN EN EL ÁREA DE SALUD,DIRIGIDA A: Panameños que deseen formarse como...,https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/doctorado-de-invest...
245,PROGRAMA DE BECAS FULBRIGHT-SENACYT-IFARHU En ...,DIRIGIDO A: Profesionales panameños residentes...,https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/programa-de-becas-f...
246,CONVOCATORIA PÚBLICA PARA EL FOMENTO A LA INNO...,OBJETIVO: Generar innovaciones para la educaci...,https://www.senacyt.gob.pa/wp-content/uploads/...,https://www.senacyt.gob.pa/convocatoria-public...


In [7]:
panama.to_excel('Panama.xlsx')